In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer,GenerationConfig,TrainingArguments,Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [5]:
dataset = load_dataset('knkarthick/dialogsum')

Found cached dataset csv (C:/Users/tusha/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base',torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

In [7]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _,param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params+=param.numel()
    return f" trainable model parameters: {trainable_model_params}\n all model parameters: {all_model_params}\n percentage of trainable model: {trainable_model_params*100/all_model_params}"
print(print_number_of_trainable_model_parameters(original_model))

 trainable model parameters: 247577856
 all model parameters: 247577856
 percentage of trainable model: 100.0


### test model with zero shot inference

In [10]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
    Summarize the following conversation .
    {dialogue}
    Summary :
    """
    # input constructed prompt instead of dialogue.
inputs = tokenizer(prompt,return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs['input_ids'],
        max_new_tokens=200  # tuning parameters
    )[0],
    skip_special_tokens=True
)
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print('INPUT PROMPT :')
print(prompt)
print(dash_line)
print('BASELINE HUMAN SUMMARY :')
print(summary)
print(dash_line)
print('MODEL OUTPUT - ZERO SHOT :\n' + output )

---------------------------------------------------------------------------------------------------
INPUT PROMPT :

    Summarize the following conversation .
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
    Summary :
    
--------------------------------------------------

### PERFORM FINE TUNING

In [11]:
def tokenize_function(example):
    start_prompt = 'Summarize the following converstaion.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer(prompt,padding='max_length',truncation=True,return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['summary'],padding='max_length',truncation=True,return_tensors='pt').input_ids
    return example
tokenize_datasets = dataset.map(tokenize_function,batched=True)
tokenize_datasets = tokenize_datasets.remove_columns(['id','topic','dialogue','summary'])

Loading cached processed dataset at C:\Users\tusha\.cache\huggingface\datasets\knkarthick___csv\knkarthick--dialogsum-cd36827d3490488d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-65f499592c1cac9c.arrow
Loading cached processed dataset at C:\Users\tusha\.cache\huggingface\datasets\knkarthick___csv\knkarthick--dialogsum-cd36827d3490488d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-dd436fed7ccedc48.arrow
Loading cached processed dataset at C:\Users\tusha\.cache\huggingface\datasets\knkarthick___csv\knkarthick--dialogsum-cd36827d3490488d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ecc472e1d332af3b.arrow


In [12]:
tokenize_datasets = tokenize_datasets.filter(lambda example,index : index %100 == 0,with_indices=True)

Loading cached processed dataset at C:\Users\tusha\.cache\huggingface\datasets\knkarthick___csv\knkarthick--dialogsum-cd36827d3490488d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b2419b3401bed00b.arrow
Loading cached processed dataset at C:\Users\tusha\.cache\huggingface\datasets\knkarthick___csv\knkarthick--dialogsum-cd36827d3490488d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-3f3998ebf909d025.arrow
Loading cached processed dataset at C:\Users\tusha\.cache\huggingface\datasets\knkarthick___csv\knkarthick--dialogsum-cd36827d3490488d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-8c927a7cd9ade104.arrow


In [13]:
print("Shapes of the datasets:")
print(f"Training: {tokenize_datasets['train'].shape}")
print(f"Validation: {tokenize_datasets['validation'].shape}")
print(f"Test: {tokenize_datasets['test'].shape}")

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)


### fine tune the model with preprocessed dataset

In [11]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)
trainer = Trainer(
    model = original_model,
    args=training_args,
    train_dataset=tokenize_datasets['train'],
    eval_dataset=tokenize_datasets['validation']
)

In [14]:
# trainer.train()
# this is the step for training model but will take too much time so we will take already trained model on same arguments

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogie-summary-checkpoint/

In [ ]:
!ls -alh ./flan-dialog-summary-checkpoint/pytorch_model.bin

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint",torch_dtype = torch.bfloat16)

## Evaluate the model qualitatively (human evaluation)

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
    Summarize the following conversation .
    {dialogue}
    Summary :
    """
    # input constructed prompt instead of dialogue.
input_ids = tokenizer(prompt,return_tensors='pt').input_ids

original_model_outputs = original_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200,num_beams=1)
    )
original_model_text_output = tokenizer.decode(original_model_outputs[0],skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(max_new_tokens=200,num_beams=1)
    )
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens=True)

print(dash_line)
print('BASELINE HUMAN SUMMARY :')
print(human_baseline_summary)
print(dash_line)
print('ORIGINAL MODEL :')
print(original_model_outputs)
print(dash_line)
print('INSTRUCT MODEL :')
print(instruct_model_outputs)

## evaluate the model quantitatively (with ROUGE metric)

In [14]:
rouge = evaluate.load('rouge')

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _,dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.
{dialogue}
Summary:"""
    input_ids = tokenizer(prompt,return_tensors='pt').input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0],skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries,original_model_summaries,instruct_model_summaries))

df = pd.DataFrame(zipped_summaries,columns=['human_baseline_summaries','original_model_summaries','instruct_model_summaries'])
df

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

print('ORIGINAL MODEL : \n'+original_model_results)
print('INSTRUCT MODEL : \n'+instruct_model_results)

In [ ]:
print('absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE')

improvement = (np.array(list(instruct_model_results.values()))) - np.array(list(original_model_results.values()))
for key,value in zip(instruct_model_results.key(),improvement):
    print(f'{key}: {value*100:.2f}%')

# PERFORM PARAMETER EFFICIENT FINE-TUNING (PEFT)

it reduces complexity as compared to full fine tuning by freezing some of parameters to not train.

PEFT is a Low-Rank Adaptation(LoRA) , at inference time the LoRA adapters needs to be reunited and hence it could be used for different tasks simultaneously.

In [1]:
from peft import LoraConfig,get_peft_model,TaskType

lora_config = LoraConfig(
    r=32 ,# rank
    lora_alpha = 32,
    target_modules=['q','v'],
    lora_dropout=0.05,
    bias = "none",
    task_type=TaskType.SEQ_2_SEQ_LM  #FLAN-T5
)

In [8]:
peft_model = get_peft_model(original_model,lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

 trainable model parameters: 3538944
 all model parameters: 251116800
 percentage of trainable model: 1.4092820552029972


In [15]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,     # higher leaning rate make it full fine tuning
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenize_datasets['train']
)

In [16]:
peft_trainer.train()
peft_model_path='./peft-dialogue-summary-checkpoint-local'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel,PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base',torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

peft_model = PeftModel.from_pretrained(
    model=peft_model_base,
    model_id='./peft-dialogue-summary-checkpoint-from-s3/',
    torch_dtype=torch.bfloat16,
    is_trainable=False
)


In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]
    
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

In [ ]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')